# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import matplotlib.pyplot as plt

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.sample(15)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
245,245,Smithers,54.7804,-127.1743,4.60,49,78,1.91,CA,1680469747
15,15,Tuktoyaktuk,69.4541,-133.0374,-19.00,71,20,4.63,CA,1680469393
355,355,Aksarka,66.5606,67.7975,-9.44,84,28,5.43,RU,1680469794
319,319,Hambantota,6.1241,81.1185,24.71,86,2,1.96,LK,1680469782
559,559,Ramsi,58.3066,25.5416,-3.23,92,9,3.57,EE,1680469879
425,425,Tarata,-17.6167,-66.0167,25.68,26,2,2.29,BO,1680469830
185,185,Vao,-22.6667,167.4833,25.32,81,45,2.63,NC,1680469729
184,184,Cairns,-16.9167,145.7667,25.04,100,31,0.00,AU,1680469729
286,286,Nanortalik,60.1432,-45.2371,-0.49,83,100,3.92,GL,1680469766
437,437,Bethlehem,-1.4558,-48.5044,24.02,94,75,1.03,BR,1680469836


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [ ]:
%%capture --no-display





# Display the map


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [19]:
# Narrow down cities that fit criteria and drop any results with null values
# A max temperature lower than 27 degrees but higher than 18, Wind speed less than 4.5 m/s, and  Zero cloudiness
# YOUR CODE HERE
cities_filter_df = city_data_df.loc[(city_data_df["Max Temp"] > 18) & (city_data_df["Max Temp"] < 27) & 
                                    (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)]
# Drop any rows with null values
cities_filter_df = cities_filter_df.dropna()

# Display sample data
cities_filter_df.sample(5)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
39,39,Mar Del Plata,-38.0023,-57.5575,23.01,73,0,3.60,AR,1680469597
24,24,Mount Pleasant,32.7941,-79.8626,24.54,55,0,4.12,US,1680469546
526,526,Salto,-31.3833,-57.9667,26.09,44,0,2.06,UY,1680469565
297,297,Sao Filipe,14.8961,-24.4956,25.37,58,0,2.75,CV,1680469772
173,173,Havelock,34.8791,-76.9013,18.98,52,0,2.24,US,1680469627


### Step 3: Create a new DataFrame called `hotel_df`.

In [20]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel"] = ""

# Display sample data
hotel_df.sample(5)


,City,Country,Lat,Lng,Humidity,Hotel
43,Hermanus,ZA,-34.4187,19.2345,97,
88,Lasa,IT,46.6166,10.7002,93,
13,Kapaa,US,22.0752,-159.3190,79,
473,Morehead,US,37.2711,-87.1764,40,
484,Olinda,BR,-8.0089,-34.8553,65,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
          "radius" = 10000
    # YOUR CODE HERE
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = # YOUR CODE HERE
    params["bias"] = # YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kapaa - nearest hotel: Pono Kai Resort
hilo - nearest hotel: Dolphin Bay Hotel
banda - nearest hotel: #acnindiafy21
makakilo city - nearest hotel: Embassy Suites by Hilton Oahu Kapolei
kahului - nearest hotel: Maui Seaside Hotel
gat - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
tikaitnagar - nearest hotel: No hotel found
san quintin - nearest hotel: Jardines Hotel
santa rosalia - nearest hotel: Hotel del Real
narwar - nearest hotel: No hotel found
port hedland - nearest hotel: The Esplanade Hotel
roebourne - nearest hotel: No hotel found
saint-francois - nearest hotel: Chez Lily
capoterra - nearest hotel: Rosa Hotel
stolac - nearest hotel: Bregava
guerrero negro - nearest hotel: Plaza sal paraiso


,City,Country,Lat,Lng,Humidity,Hotel Name
45,kapaa,US,22.0752,-159.3190,84,Pono Kai Resort
51,hilo,US,19.7297,-155.0900,83,Dolphin Bay Hotel
63,banda,IN,25.4833,80.3333,52,#acnindiafy21
81,makakilo city,US,21.3469,-158.0858,81,Embassy Suites by Hilton Oahu Kapolei
152,kahului,US,20.8947,-156.4700,60,Maui Seaside Hotel
197,gat,IL,31.6100,34.7642,100,No hotel found
211,laguna,US,38.4210,-121.4238,79,Holiday Inn Express & Suites
240,tikaitnagar,IN,26.9500,81.5833,59,No hotel found
265,san quintin,MX,30.4833,-115.9500,74,Jardines Hotel
340,santa rosalia,MX,27.3167,-112.2833,56,Hotel del Real


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)